# Laptop Recommendation Assistant
This Python script serves as a basic interactive laptop recommendation assistant using OpenAI's API. The assistant gathers user requirements and provides tailored recommendations based on the input. Below is a detailed explanation of the various components and functionalities of the code.




### 1. Importing Libraries
### 2. Setting Display Options --> pd.set_option('display.width', 100)
### 3. Reading the Dataset

In [9]:
import os
import pandas as pd
from IPython.display import display, HTML
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('laptop.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


### 4. OpenAI API Key Management
The OpenAI API key is read from a text file and set for use in API calls. It's also stored in the environment variables for broader access within the application

In [10]:
# Read the OpenAI API key
openai.api_key = open("OpenAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

## Part 1 : Intent Clarity Layer & Intent Confirmation Layer
#### Initialize conversation : This function sets up the initial conversation parameters and guidelines for the assistant.
Example User Requirement Dictionary

example_user_dict = {
    'GPU intensity': "high",
    'Display quality': "high",
    'Portability': "low",
    'Multitasking': "high",
    'Processing speed': "high",
    'Budget': "150000"
}


In [11]:
def initialize_conversation():
    """Initialize the conversation with the bot by providing system instructions and example user requirements.

    Returns:
        list: A list of dictionaries where each dictionary contains the role ("system") and the system message content.
    """
    delimiter = '####'  # Define your delimiter for structuring instructions
    
    # Example user dictionary to guide the assistant in the conversation
    example_user_dict = {
        'GPU intensity': "high",
        'Display quality': "high",
        'Portability': "low",
        'Multitasking': "high",
        'Processing speed': "high",
        'Budget': "150000"
    }
    
    # Template for the dictionary that the assistant will fill
    example_user_req = {
        'GPU intensity': "_",
        'Display quality': "_",
        'Portability': "_",
        'Multitasking': "_",
        'Processing speed': "_",
        'Budget': "_"
    }
    
    # Instructions and system message to the assistant
    system_message = f"""
    Welcome! You are an expert laptop advisor, trusted for your deep knowledge of laptops. Users come to you to find the most suitable laptops based on their specific needs.

    Your goal is to have a conversation with the user and fill out their laptop requirements based on their responses. 
    The final aim is to correctly fill in a dictionary with the following keys: 'GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', and 'Budget'.
    
    The structure of the dictionary should look like this:
    {example_user_req}
    
    {delimiter}
    ### Instructions
    - All fields except 'Budget' should be filled with values: 'low', 'medium', or 'high' based on the user's priorities.
    - The 'Budget' field should be filled with a numeric value.
    - The budget value must be **above or equal to 25,000 INR**. If the user provides a lower value:
      - First, respond with: "You will not get a hair straightener in this budget. Please increase your budget, and I will assist."
      - If the user continues to provide a budget below 25,000 INR, respond with: "We do not offer laptops in this price range."
    
    - Never assign values randomly. Always base them on the user's responses.
    - Ask clarifying questions if the user's input is unclear or insufficient.

    {delimiter}
    ### Conversation Flow
    1. Start by welcoming the user and asking for their requirements.
    2. If the user is unsure about their primary requirement, guide the conversation by asking targeted questions, referring to the following keys: {list(example_user_dict.keys())}.
    3. Fill the fields confidently based on the user's responses.
    4. Ensure each field is filled correctly; if not, ask additional questions.

    {delimiter}
    ### Example Conversation
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop with high multitasking capability and a high-quality display for editing. Could you confirm if you're more focused on video editing, photo editing, or both?"
    
    User: "I primarily work with After Effects."
    Assistant: "Thank you! Since After Effects involves graphic rendering, you'll need a high GPU. Do you work with 4K videos or other large media files?"
    
    User: "Yes, I work with 4K videos."
    Assistant: "Noted. You will also need high processing speed and sufficient storage. Could you also let me know if portability is important to you, or if you primarily work in one location?"
    
    User: "I usually work from a stationary location."
    Assistant: "Got it. Finally, what is your budget for the laptop?"
    
    User: "My max budget is 1.5 lakh INR."
    
    Assistant: {example_user_dict}

    {delimiter}
    Always start the conversation with a friendly welcome!
    """
    
    # Return the system's starting message
    conversation = [{"role": "system", "content": system_message}]
    return conversation



In [12]:
debug_conversation = initialize_conversation()
print(debug_conversation)
print(debug_conversation[0]['content'])

[{'role': 'system', 'content': '\n    Welcome! You are an expert laptop advisor, trusted for your deep knowledge of laptops. Users come to you to find the most suitable laptops based on their specific needs.\n\n    Your goal is to have a conversation with the user and fill out their laptop requirements based on their responses. \n    The final aim is to correctly fill in a dictionary with the following keys: \'GPU intensity\', \'Display quality\', \'Portability\', \'Multitasking\', \'Processing speed\', and \'Budget\'.\n    \n    The structure of the dictionary should look like this:\n    {\'GPU intensity\': \'_\', \'Display quality\': \'_\', \'Portability\': \'_\', \'Multitasking\': \'_\', \'Processing speed\': \'_\', \'Budget\': \'_\'}\n    \n    ####\n    ### Instructions\n    - All fields except \'Budget\' should be filled with values: \'low\', \'medium\', or \'high\' based on the user\'s priorities.\n    - The \'Budget\' field should be filled with a numeric value.\n    - The budg

##  Generating Chat Completions 
This function interacts with OpenAI's chat model to get responses based on the ongoing conversation. It uses the tenacity library to retry the request in case of failures.

In [13]:
# lets look into get_chat_complition()
# this function will take the ongoing conversation as the input and returns the response by the assistance, we ll use the chat completion
# function to perform llm calls to openAI

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [14]:
input_prompt = 'what is the capital of manipur india just one word please'
messages = [{'role':'user','content':input_prompt}]
get_chat_completions(messages)
get_chat_completions(messages,json_format = True)


{'output': 'Imphal'}

### We are doing self consistancy prompting here to get the accurate result , will do it for  6 times
iterate_llm_response 

In [15]:
def iterate_llm_response(funct, debug_response, num = 6):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
iterate_llm_response(get_chat_completions, messages)

Iteration: 0
Imphal

{
  "output": "Imphal"
}
--------------------------------------------------
Iteration: 1
{
    "output": "Imphal"
}
--------------------------------------------------
Iteration: 2
{
    "output": "Imphal"
}
--------------------------------------------------
Iteration: 3
{
    "output": "Imphal"
}
--------------------------------------------------
Iteration: 4
{
    "output": "Imphal"
}
--------------------------------------------------
Iteration: 5
{
    "output": "Imphal"
}
--------------------------------------------------


## Gathering Laptop Requirements
This function engages users in conversation to gather their laptop requirements.
Collecting User Requirements
The assistant asks the user for various requirements and fills a dictionary based on their responses. Each attribute (GPU intensity, display quality, portability, etc.) is gathered through a series of input statements, utilizing the determine_level function to categorize user inputs into "high," "medium," or "low."


In [16]:
def gather_laptop_requirements_with_autofill():
    """
    This function engages in a conversation with the user, starting with an introduction,
    and gathers laptop requirements, automatically filling in values based on user input.

    Returns:
        dict: A dictionary containing the user's laptop preferences.
    """

    # Initialize the conversation
    print("Welcome! Let's find the perfect laptop for your needs.")

    # Ask for the user's introduction
    user_input = input("Please introduce yourself and share a bit about what you primarily need the laptop for: ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return

    print(f"Thanks for introducing yourself! You mentioned: '{user_input}'")

    # Initialize the empty requirements dictionary
    user_req = {
        'GPU intensity': "_",
        'Display quality': "_",
        'Portability': "_",
        'Multitasking': "_",
        'Processing speed': "_",
        'Budget': "_"
    }

    # Define a function to determine the level based on keywords
    def determine_level(user_input):
        input_lower = user_input.lower()
        if any(keyword in input_lower for keyword in ["high", "very", "excellent", "great"]):
            return "high"
        elif any(keyword in input_lower for keyword in ["medium", "average", "decent"]):
            return "medium"
        else:
            return "low"

    # Transition into the requirements gathering
    print("\nLet's dive into your laptop requirements. I'll ask you a few questions to better understand your needs.")

    # Gather GPU intensity
    user_input = input("How would you rate the importance of GPU intensity for your tasks (please type high/medium/low or describe): ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return
    user_req['GPU intensity'] = determine_level(user_input)

    # Gather Display quality
    user_input = input("What level of display quality do you prefer (please type high/medium/low or describe): ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return
    user_req['Display quality'] = determine_level(user_input)

    # Gather Portability
    user_input = input("How important is portability for you (please type high/medium/low or describe): ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return
    user_req['Portability'] = determine_level(user_input)

    # Gather Multitasking capability
    user_input = input("What level of multitasking capability do you need (please type high/medium/low or describe): ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return
    user_req['Multitasking'] = determine_level(user_input)

    # Gather Processing speed
    user_input = input("How important is processing speed for your use case (please type high/medium/low or describe): ")
    if user_input.lower() == "exit":
        print("Thank you for your time. Goodbye!")
        return
    user_req['Processing speed'] = determine_level(user_input)

    # Gather Budget
    while True:
        try:
            user_input = input("Lastly, what's your budget in INR (must be >= 25,000)? ")
            if user_input.lower() == "exit":
                print("Thank you for your time. Goodbye!")
                return
            budget_value = int(user_input)
            if budget_value >= 25000:
                user_req['Budget'] = budget_value
                break
            else:
                print("You will not get a good laptop in this budget. Please increase your budget.")
        except ValueError:
            print("Please enter a valid numeric value for budget.")

    # Print the final dictionary of user requirements
    print("\nThank you! Here are the requirements you provided:")
    print(user_req)

    # Return the dictionary of user requirements
    return user_req

# Example usage
laptop_requirements = gather_laptop_requirements_with_autofill()




Welcome! Let's find the perfect laptop for your needs.
Thanks for introducing yourself! You mentioned: 'i am Arihant and i am coder'

Let's dive into your laptop requirements. I'll ask you a few questions to better understand your needs.

Thank you! Here are the requirements you provided:
{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 50000}


## Moderation Check Function
This function checks user inputs against OpenAI’s moderation API to ensure they comply with community standards.

In [17]:
# This function will be called as moderation_check  this will be default function in any llm project by calling moderation in from opeai
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if moderation_output == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [18]:
moderation_check("i am a coder")

'Not Flagged'

## Iterating Responses for Consistency Testing
This function repeatedly calls a specified function to test the consistency of its responses.


In [19]:
# get_chat_completions(laptop_requirements)
def iterate_llm_response(funct, debug_response, num = 5):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer



### Part 1 : Intent confirmation layer

In [20]:
import openai
import json

def intent_confirmation_layer(response_assistant):
    allowed_values = {'low', 'medium', 'high'}

    prompt = f"""
    You are a senior evaluator with an eye for detail. You are provided with a user's input, and your task is to evaluate it based on 6 keys:
    {{
    'GPU intensity': 'values',
    'Display quality': 'values',
    'Portability': 'values',
    'Multitasking': 'values',
    'Processing speed': 'values',
    'Budget': 'number'
    }}
    The values for the first five keys must be one of: {allowed_values}. The 'Budget' key should be a numerical value but that can be also under quotes
    like it can 15000 or '15000'.
    
    Return a one-word string in JSON format at the key 'result' as 'Yes' if all the values are valid, otherwise 'No'.
    If the answer is 'No', include a second key 'reason' explaining why the validation failed.
    """

    # Construct the messages for the model
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"Here is the input: {response_assistant}"}
    ]

    # Request completion from the OpenAI API
    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages
                                   # ,response_format={ "type": "json_object" },
                                   # seed = 1234
                                    # n = 5
                                    )
    # Extract the content and load as JSON
    json_output = json.loads(response.choices[0].message.content)

    return json_output


In [21]:
laptop_requirements

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'high',
 'Multitasking': 'high',
 'Processing speed': 'high',
 'Budget': 50000}

In [22]:
response = intent_confirmation_layer(laptop_requirements)
response.get('result') # Extract the result key from the dictionary

'Yes'

Stage 2 , Implementing the product mapping and information extractions

In [23]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Display Type, Screen Resolution, Display Size)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM Size)",
        "Processing speed":"(CPU Type, Core, Clock Speed)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high, this has to be follwoed in any circumstances , i dont want to see any value apart from these low/medium and high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}. and again nothing in apart from high low and medium"""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    return response

In [24]:
##  a new csv with where i will apply above function and use create a new field with laptop_features
laptop_df= pd.read_csv('laptop.csv')
## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))
pd.set_option('display.max_colwidth', None)

In [25]:
for i in  laptop_df['laptop_feature']:
    print(i)

{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high'}
{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high'}
{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'low'}
{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high'}
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'high'}
{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'low', 'Processing speed': 'medium'}
{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'high', 'Multitaski

In [27]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

In [ ]:
# #def compare_laptops_with_user(response_dict_n):

# # Users budget
# response_dict_n= laptop_requirements 
# Budget = response_dict_n['Budget'] 

# #The Corpus data 
# laptop_df= pd.read_csv('updated_laptop.csv')

# # Changing the datatype of laptop_feature to dictionary in the field
# laptop_df['laptop_feature'] = laptop_df['laptop_feature'].apply(ast.literal_eval)

# # Creating a new copy of the corpus dataframe
# filtered_laptops= laptop_df.copy()

# # Changing the datatype of Price from the corpus dataframe
# filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)

# # Lets restrict the budgeted laptop itself in first place
# filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
# filtered_laptops

# # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
# mappings = {'low': 0, 'medium': 1, 'high': 2}

# # Initializing the score to 0 for now 
# filtered_laptops['Score'] = 0

# for index, row in filtered_laptops.iterrows():
#     user_product_match_str = row['laptop_feature']
#     laptop_values_in_csv = user_product_match_str
#     score = 0

#     for key, value in  response_dict_n.items():
#         if key =='Budget':
#             continue 
#         laptop_value = laptop_values_in_csv.get(key, None) 
#         laptop_mapping = mappings.get(laptop_value, -1)
#         user_mapping = mappings.get(value,-1)
#         if laptop_mapping>=user_mapping:
#             score +=1
#     filtered_laptops.loc[index, 'Score'] = score



In [ ]:
# top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
# top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
# top_laptops_json = top_laptops.to_json(orient='records') 
# return top_laptops_json

In [28]:
laptop_requirements

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'high',
 'Multitasking': 'high',
 'Processing speed': 'high',
 'Budget': 50000}

In [50]:
def compare_laptops_with_user(response_dict_n):
    # Users budget
    budget = response_dict_n.get('Budget', 0)

    # The Corpus data
    laptop_df = pd.read_csv('updated_laptop.csv')

    # Changing the datatype of 'laptop_feature' to dictionary
    laptop_df['laptop_feature'] = laptop_df['laptop_feature'].apply(ast.literal_eval)

    # Creating a new copy of the corpus dataframe
    filtered_laptops = laptop_df.copy()

    # Changing the datatype of Price from the corpus dataframe
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)

    # Restrict the budgeted laptops
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {'low': 0, 'medium': 1, 'high': 2}

    # Initialize the score to 0
    filtered_laptops['Score'] = 0

    # Iterate through the filtered laptops and calculate the score
    for index, row in filtered_laptops.iterrows():
        laptop_values_in_csv = row['laptop_feature']
        score = 0

        # Compare each feature between user response and laptop feature
        for key, value in response_dict_n.items():
            if key == 'Budget':
                continue  # Skip Budget comparison

            laptop_value = laptop_values_in_csv.get(key, None)
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(value, -1)

            if laptop_mapping >= user_mapping:
                score += 1
        #print(score)
        # Update the score for the laptop
        filtered_laptops.loc[index, 'Score'] = score

    # Drop the 'laptop_feature' column and sort by 'Score'
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    #top_laptops.to_csv("new_updated_laptop.csv",index=False,header = True)
    top_laptops = top_laptops.sort_values('Score', ascending=False)

    # Convert the top laptops to JSON format
    top_laptops_json = top_laptops.to_json(orient='records')

    return top_laptops_json


In [51]:
a= compare_laptops_with_user(laptop_requirements)
list_from_string = ast.literal_eval(a)
for i in list_from_string:
    print(i)

{'Brand': 'Dell', 'Model Name': 'Inspiron', 'Core': 'i5', 'CPU Manufacturer': 'Intel', 'Clock Speed': '2.4 GHz', 'RAM Size': '8GB', 'Storage Type': 'SSD', 'Display Type': 'LCD', 'Display Size': '15.6"', 'Graphics Processor': 'Intel UHD', 'Screen Resolution': '1920x1080', 'OS': 'Windows 10', 'Laptop Weight': '2.5 kg', 'Special Features': 'Backlit Keyboard', 'Warranty': '1 year', 'Average Battery Life': '6 hours', 'Price': 35000, 'Description': 'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU f

product_validation_layer():

In [52]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] >= 1:
            data1.append(data[i])

    return data1

In [53]:
saving_the_result  = compare_laptops_with_user(laptop_requirements)
recommendation_validation(saving_the_result)

[{'Brand': 'Dell',
  'Model Name': 'Inspiron',
  'Core': 'i5',
  'CPU Manufacturer': 'Intel',
  'Clock Speed': '2.4 GHz',
  'RAM Size': '8GB',
  'Storage Type': 'SSD',
  'Display Type': 'LCD',
  'Display Size': '15.6"',
  'Graphics Processor': 'Intel UHD',
  'Screen Resolution': '1920x1080',
  'OS': 'Windows 10',
  'Laptop Weight': '2.5 kg',
  'Special Features': 'Backlit Keyboard',
  'Warranty': '1 year',
  'Average Battery Life': '6 hours',
  'Price': 35000,
  'Description': 'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additio